# Quality Control and Trimming

This notebook will go through the workflow for read quality control and trimming. We will follow each of the steps below, that will require time on the HPC to run. Be sure to check back after each step to make sure you have the right files, and start the next step.   

1. Quality control using fastqc to determine quality thresholds.
2. Compressing files before trimming.
3. Trimming reads with [Trimmomatic](https://carpentries-lab.github.io/metagenomics-analysis/03-trimming-filtering/index.html).
4. Optional: Final QC check after trimming


## Getting Started

You will need to rerun this section each time you come back to this notebook to kick off the next step.

In [ ]:
# set the variables for your netid and xfile
netid = "kolodisner"
xfile = "xac"

In [ ]:
# Go into the working directory
work_dir = "/xdisk/bhurwitz/virus_hunting/" + netid + "/02_qc_trimming"
%cd $work_dir

## Creating a config file
Each of the scripts below executes code that requires certain variables to be set. So we don't need to edit the code in each of the scripts, we are going to use a config file that defines all of these variables. Then when we want to use these variables in the script, we will "source" the config file to set the variables. This is generally a good practice in writing scripts on the HPC, that makes it so you only need to modify the config file (rather than each individual script). We are going to create this file using the variables you set above in "Getting started". Note that you only need to create this config file once, even if you are returning to complete the next step.   

In [3]:
# create a config file with all of the variables you need
!echo "export NETID=$netid" > config.sh
!echo "export XFILE=$xfile" >> config.sh
!echo "export XFILE_DIR=/xdisk/bhurwitz/virus_hunting/$netid" >> config.sh
!echo "export WORK_DIR=/xdisk/bhurwitz/virus_hunting/$netid/02_qc_trimming" >> config.sh
!echo "export FASTQ_DIR=/xdisk/bhurwitz/virus_hunting/$netid/01_get_fastq" >> config.sh

In [8]:
# check the config file to be sure it is correct
# Is your netid and xfile correct? Do you have the right directories?
!cat config.sh

export NETID=kolodisner
export XFILE=xac
export XFILE_DIR=/xdisk/bhurwitz/virus_hunting/kolodisner
export WORK_DIR=/xdisk/bhurwitz/virus_hunting/kolodisner/02_qc_trimming
export FASTQ_DIR=/xdisk/bhurwitz/virus_hunting/kolodisner/01_get_fastq


## Step 1: Assessing Read Quality

Now that we have all of our data downloaded, we are ready to start the quality control process. We will use a tool called fastqc that generates a report about the quality of our sequence data.

First, we will create an sbatch script that runs fastqc on each of the sequence files. Note that when you kick off this analysis by running "sbatch" below, you will need to wait ~1 hour for the results to come back, depending on the queue wait time. 

In [5]:
# Create a script to run fastqc on each of our accessions
# A few important points:
# 1. We are using the variables from the config file via
# the `source ./config.sh` command in the script.
# 2. fastqc runs on each of the fastq files in the $FASTQ_DIR
# 3. We are creating a directory called check_fastqc in our home directory
# this allows us to copy the *html files produced by fastqc and explore
# them using Jupyter server on the on demand hpc portal.
# 4. The fastqc program runs in the $FASTQ_DIR, but to keep our files
# organized, we are going to move the results into our $WORK_DIR.
my_code = '''#!/bin/bash
#SBATCH --ntasks=1
#SBATCH --nodes=1             
#SBATCH --time=10:00:00   
#SBATCH --partition=standard
#SBATCH --account=bhurwitz
#SBATCH --array=0-2                         
#SBATCH --output=Job-fastqc-%a.out
#SBATCH --cpus-per-task=1                  
#SBATCH --mem=4G                           

pwd; hostname; date

source ./config.sh
names=($(cat $XFILE_DIR/$XFILE))

apptainer run /contrib/singularity/shared/bhurwitz/fastqc-0.11.9.sif fastqc \
    $FASTQ_DIR/${names[${SLURM_ARRAY_TASK_ID}]}_*.fastq*

mkdir ~/check_fastqc
cp $FASTQ_DIR/${names[${SLURM_ARRAY_TASK_ID}]}_*_fastqc.html ~/check_fastqc 
mv $FASTQ_DIR/${names[${SLURM_ARRAY_TASK_ID}]}_*_fastqc.html $WORK_DIR
mv $FASTQ_DIR/${names[${SLURM_ARRAY_TASK_ID}]}_*_fastqc.zip $WORK_DIR
 
'''

with open('run_fastqc.sh', mode='w') as file:
    file.write(my_code)

In [6]:
# Check the code and make sure your script above was created.
!cat run_fastqc.sh

#!/bin/bash
#SBATCH --ntasks=1
#SBATCH --nodes=1             
#SBATCH --time=10:00:00   
#SBATCH --partition=standard
#SBATCH --account=bhurwitz
#SBATCH --array=0-2                         
#SBATCH --output=Job-fastqc-%a.out
#SBATCH --cpus-per-task=1                  
#SBATCH --mem=4G                           

pwd; hostname; date

source ./config.sh
names=($(cat $XFILE_DIR/$XFILE))

apptainer run /contrib/singularity/shared/bhurwitz/fastqc-0.11.9.sif fastqc     $FASTQ_DIR/${names[${SLURM_ARRAY_TASK_ID}]}_*.fastq*

mkdir ~/check_fastqc
cp $FASTQ_DIR/${names[${SLURM_ARRAY_TASK_ID}]}_*_fastqc.html ~/check_fastqc 
mv $FASTQ_DIR/${names[${SLURM_ARRAY_TASK_ID}]}_*_fastqc.html $WORK_DIR
mv $FASTQ_DIR/${names[${SLURM_ARRAY_TASK_ID}]}_*_fastqc.zip $WORK_DIR
 


In [5]:
# you should be in your working directory when you run this script
# do you see your config.sh file, and the fastqc_parallel.sh script?
!pwd
!ls

/xdisk/bhurwitz/virus_hunting/kolodisner/02_qc_trimming
ERR2198703_1_fastqc.html  ERR2198705_1_fastqc.html  Job-gzip-1.out
ERR2198703_1_fastqc.zip   ERR2198705_1_fastqc.zip   Job-gzip-2.out
ERR2198703_2_fastqc.html  ERR2198705_2_fastqc.html  config.sh
ERR2198703_2_fastqc.zip   ERR2198705_2_fastqc.zip   run_fastqc.sh
ERR2198704_1_fastqc.html  Job-fastqc-0.out	    run_gzip_untrimmedfiles.sh
ERR2198704_1_fastqc.zip   Job-fastqc-1.out	    trimmed_reads
ERR2198704_2_fastqc.html  Job-fastqc-2.out	    unpaired_reads
ERR2198704_2_fastqc.zip   Job-gzip-0.out


In [8]:
# Let's run sbatch to run fastqc on each of the FASTQ files
# Remember that this may take 1 hour to run, so take a break, 
# and get a coffee.
!sbatch ./run_fastqc.sh

Submitted batch job 2973607


In [15]:
# You can check if it is running using the squeue command
# If you get an error "Invalid job id specified", then your job already
# completed.
# You can also check for all jobs under your netid
# !squeue --job=$netid
!squeue --job=MY_JOBID

squeue: error: Invalid job id: MY_JOBID


In [19]:
# Once your jobs have run (or are running) you can check the progress
# and also look for errors in the *out files
# For example, you can look at Job-fastqc-0.out
!ls
!cat Job-fastqc-0.out

ERR2198703_1_fastqc.html  ERR2198704_2_fastqc.html  Job-fastqc-0.out
ERR2198703_1_fastqc.zip   ERR2198704_2_fastqc.zip   Job-fastqc-1.out
ERR2198703_2_fastqc.html  ERR2198705_1_fastqc.html  Job-fastqc-2.out
ERR2198703_2_fastqc.zip   ERR2198705_1_fastqc.zip   config.sh
ERR2198704_1_fastqc.html  ERR2198705_2_fastqc.html  run_fastqc.sh
ERR2198704_1_fastqc.zip   ERR2198705_2_fastqc.zip
/xdisk/bhurwitz/virus_hunting/kolodisner/02_qc_trimming
i16n0.ocelote.hpc.arizona.edu
Thu Mar 28 11:15:18 MST 2024
Started analysis of ERR2198703_1.fastq
Approx 5% complete for ERR2198703_1.fastq
Approx 10% complete for ERR2198703_1.fastq
Approx 15% complete for ERR2198703_1.fastq
Approx 20% complete for ERR2198703_1.fastq
Approx 25% complete for ERR2198703_1.fastq
Approx 30% complete for ERR2198703_1.fastq
Approx 35% complete for ERR2198703_1.fastq
Approx 40% complete for ERR2198703_1.fastq
Approx 45% complete for ERR2198703_1.fastq
Approx 50% complete for ERR2198703_1.fastq
Approx 55% complete for ERR21987

In [20]:
# Double check that all of your files have run through fastqc.
# Do you see a *.html and *.zip file for each one?
!ls /xdisk/bhurwitz/virus_hunting/$netid/02_qc_trimming

ERR2198703_1_fastqc.html  ERR2198704_2_fastqc.html  Job-fastqc-0.out
ERR2198703_1_fastqc.zip   ERR2198704_2_fastqc.zip   Job-fastqc-1.out
ERR2198703_2_fastqc.html  ERR2198705_1_fastqc.html  Job-fastqc-2.out
ERR2198703_2_fastqc.zip   ERR2198705_1_fastqc.zip   config.sh
ERR2198704_1_fastqc.html  ERR2198705_2_fastqc.html  run_fastqc.sh
ERR2198704_1_fastqc.zip   ERR2198705_2_fastqc.zip


Great job! It looks like your files have all been checked with fastqc. Before moving on to the next step, start up the Jupyter server on HPC on demand, navigate to the folder called check_fastqc, and double click on each of the html files to check the quality of each of your sequence files. Be sure to refer back to the in-class exercise on quality control to understand what each of the sections means. Because your sequence data come from the SRA, you will likely find that your files are all passing quality control checks already. But, to be certain, we will run a few basic "screening and cleaning" steps via trimmomatic in Step 3 to make sure the sequences are up to par. 

## Step 2: Compressing your *fastq files using gzip 

Trimmomatic works on FASTQ files that are compressed with either gzip or bzip2. So, before we can run trimmomatic, we will need to compress our read files. We'll be using gzip to compress files and get the .gz file extension we need. These FASTQ files are massive, and gzip takes time to run, so lets create a script to sbatch the compression job.

In [21]:
# Let's create a script that gzip's all of the FASTQ files
# These are huge files, so it may take 2 hours to run.
# This script uses gzip to compress each of the *.fastq files.
my_code = '''#!/bin/bash
#SBATCH --ntasks=1
#SBATCH --nodes=1             
#SBATCH --time=10:00:00   
#SBATCH --partition=standard
#SBATCH --account=bhurwitz
#SBATCH --array=0-2             # the number of accession files
#SBATCH --output=Job-gzip-%a.out
#SBATCH --cpus-per-task=1        # num CPUs per task
#SBATCH --mem=4G                 # total memory per node
 
pwd; hostname; date
source ./config.sh
names=($(cat ${XFILE_DIR}/${XFILE}))
gzip ${FASTQ_DIR}/${names[${SLURM_ARRAY_TASK_ID}]}_*.fastq
'''

with open('run_gzip_untrimmedfiles.sh', mode='w') as file:
    file.write(my_code)

In [25]:
# Let's double check that your script was created.
!cat run_gzip_untrimmedfiles.sh

#!/bin/bash
#SBATCH --ntasks=1
#SBATCH --nodes=1             
#SBATCH --time=10:00:00   
#SBATCH --partition=standard
#SBATCH --account=bhurwitz
#SBATCH --array=0-2             # the number of accession files
#SBATCH --output=Job-gzip-%a.out
#SBATCH --cpus-per-task=1        # num CPUs per task
#SBATCH --mem=4G                 # total memory per node
 
pwd; hostname; date
source ./config.sh
names=($(cat ${XFILE_DIR}/${XFILE}))
gzip ${FASTQ_DIR}/${names[${SLURM_ARRAY_TASK_ID}]}_*.fastq


In [31]:
# OK, we are ready to kick off the script
# Time to go get another coffee...before completing Step 3.
!sbatch run_gzip_untrimmedfiles.sh

Submitted batch job 2982576


In [33]:
# You can check if it is running using the squeue command or looking for
# jobs under your netid
!squeue --user=$netid

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           2982486  standard  jupyter kolodisn  R       5:07      1 i16n0
           2981033  standard  jupyter kolodisn  R    1:28:15      1 i16n0


In [30]:
# Check to see if all of your *.fastq files are gzip-ed
# Note that these files are in the "02_qc_trimming" directory
# You should just see one gzipped file per fastq
# For example, ERR2198631_1.fastq.gz not ERR2198631_1.fastq too...
# If you see both .fastq and .fastq.gz the gzip command 
# is still in progress.
!ls /xdisk/bhurwitz/virus_hunting/$netid/02_qc_trimming

ERR2198703_1_fastqc.html  ERR2198705_1_fastqc.html  Job-gzip-1.out
ERR2198703_1_fastqc.zip   ERR2198705_1_fastqc.zip   Job-gzip-2.out
ERR2198703_2_fastqc.html  ERR2198705_2_fastqc.html  config.sh
ERR2198703_2_fastqc.zip   ERR2198705_2_fastqc.zip   run_fastqc.sh
ERR2198704_1_fastqc.html  Job-fastqc-0.out	    run_gzip_untrimmedfiles.sh
ERR2198704_1_fastqc.zip   Job-fastqc-1.out	    trimmed_reads
ERR2198704_2_fastqc.html  Job-fastqc-2.out	    unpaired_reads
ERR2198704_2_fastqc.zip   Job-gzip-0.out


## Step 3: Trimming .fastq Files

In order to run trimmomatic in a PE (paired-end) format we'll need two files. In our case, we have *_1.fastq.gz and *_2.fastq.gz for each accession from the SRA. You should now have those from the steps above.

Note that we are following the same trimming protocol from the in-class exercise.

### Initial Data Management
The output from trimmomatic will give us 4 output files (forward paired, forward unpaired, reverse paired and reverse unpaired. To keep our data organized, let's set up some output directories so the script can organize our data as it runs.


In [20]:
# Create the trimmed and unpaired directories
import os

trim_dir = work_dir + "/trimmed_reads"
unpair_dir = work_dir + "/unpaired_reads"

if os.path.isdir(trim_dir):
    print("trim_dir exists")
else:
    os.mkdir(trim_dir)

if os.path.isdir(unpair_dir):
    print("unpair_dir exists")
else:
    os.mkdir(unpair_dir)

In [3]:
# we need to copy the adapter file into your current working directory
!cp /xdisk/bhurwitz/virus_hunting/python_notebooks/TruSeq3-PE-2.fa .  

In [4]:
# Let's create a script that runs trimmomatic on all of our fastq files
# you can only run this after the *.fastq files are all gzip-ed (step 2)
my_code = '''#!/bin/bash
#SBATCH --ntasks=1
#SBATCH --nodes=1             
#SBATCH --time=10:00:00   
#SBATCH --partition=standard
#SBATCH --account=bhurwitz
#SBATCH --array=0-2                          # the number of accessions
#SBATCH --output=Job-trim-%a.out
#SBATCH --cpus-per-task=1                    # num CPUs per task
#SBATCH --mem=4G                             # total memory per node
 
pwd; hostname; date
source ./config.sh
names=($(cat ${XFILE_DIR}/${XFILE}))

TRIM_DIR="${WORK_DIR}/trimmed_reads"
UNPAIR_DIR="${WORK_DIR}/unpaired_reads"

apptainer run /contrib/singularity/shared/bhurwitz/trimmomatic:0.39--hdfd78af_2.sif trimmomatic PE -phred33 \
    ${FASTQ_DIR}/${names[${SLURM_ARRAY_TASK_ID}]}_1.fastq.gz ${FASTQ_DIR}/${names[${SLURM_ARRAY_TASK_ID}]}_2.fastq.gz \
    ${TRIM_DIR}/${names[${SLURM_ARRAY_TASK_ID}]}_1.fastq.gz ${UNPAIR_DIR}/${names[${SLURM_ARRAY_TASK_ID}]}_1.fastq.gz \
    ${TRIM_DIR}/${names[${SLURM_ARRAY_TASK_ID}]}_2.fastq.gz ${UNPAIR_DIR}/${names[${SLURM_ARRAY_TASK_ID}]}_2.fastq.gz \
    ILLUMINACLIP:TruSeq3-PE-2.fa:2:30:10 SLIDINGWINDOW:4:20
'''

with open('run_trimmomatic_fastq.sh', mode='w') as file:
    file.write(my_code)

In [5]:
# Did you create the script file correctly?
!cat run_trimmomatic_fastq.sh

#!/bin/bash
#SBATCH --ntasks=1
#SBATCH --nodes=1             
#SBATCH --time=10:00:00   
#SBATCH --partition=standard
#SBATCH --account=bhurwitz
#SBATCH --array=0-2                          # the number of accessions
#SBATCH --output=Job-trim-%a.out
#SBATCH --cpus-per-task=1                    # num CPUs per task
#SBATCH --mem=4G                             # total memory per node
 
pwd; hostname; date
source ./config.sh
names=($(cat ${XFILE_DIR}/${XFILE}))

TRIM_DIR="${WORK_DIR}/trimmed_reads"
UNPAIR_DIR="${WORK_DIR}/unpaired_reads"

apptainer run /contrib/singularity/shared/bhurwitz/trimmomatic:0.39--hdfd78af_2.sif trimmomatic PE -phred33     ${FASTQ_DIR}/${names[${SLURM_ARRAY_TASK_ID}]}_1.fastq.gz ${FASTQ_DIR}/${names[${SLURM_ARRAY_TASK_ID}]}_2.fastq.gz     ${TRIM_DIR}/${names[${SLURM_ARRAY_TASK_ID}]}_1.fastq.gz ${UNPAIR_DIR}/${names[${SLURM_ARRAY_TASK_ID}]}_1.fastq.gz     ${TRIM_DIR}/${names[${SLURM_ARRAY_TASK_ID}]}_2.fastq.gz ${UNPAIR_DIR}/${names[${SLURM_ARRAY_TASK_ID}]}_2.fastq.

In [23]:
# Now we can run trimmomatic
!sbatch run_trimmomatic_fastq.sh

Submitted batch job 3004157


In [27]:
# You can check if it is running using the squeue command
!squeue --user=$netid

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
         3004157_0  standard run_trim kolodisn  R       2:59      1 i16n7
         3004157_1  standard run_trim kolodisn  R       2:59      1 i16n7
         3004157_2  standard run_trim kolodisn  R       2:59      1 i16n10
         3003643_0  standard run_trim kolodisn  R      35:34      1 i16n1
         3003643_1  standard run_trim kolodisn  R      35:34      1 i16n2
         3003643_2  standard run_trim kolodisn  R      35:34      1 i16n2
           3003641  standard  jupyter kolodisn  R      40:35      1 i16n2


### Checking your output files

Once your job has completed, you should see that there are four output files from two input files. The trimmomatic program places all of the "orphaned" reads in a separate file from the trimmed reads. Reads can become orphaned when their "mate pair" is either too short, or too low quality. For our analyses going forward, we will only use the reads that were trimmed, and have both the forward and reverse read. 

In [25]:
# Let's check the file sizes to see that they are smaller for our trimmed
# reads
!echo "trimmed:"
!ls -l /xdisk/bhurwitz/virus_hunting/$netid/02_qc_trimming/trimmed_reads/*fastq.gz
!echo "untrimmed:"
!ls -l /xdisk/bhurwitz/virus_hunting/$netid/01_get_fastq/*fastq.gz

trimmed:
-rw-r--r-- 1 kolodisner bhurwitz 1909334091 Mar 29 12:43 /xdisk/bhurwitz/virus_hunting/kolodisner/02_qc_trimming/trimmed_reads/ERR2198703_1.fastq.gz
-rw-r--r-- 1 kolodisner bhurwitz 1930343936 Mar 29 12:43 /xdisk/bhurwitz/virus_hunting/kolodisner/02_qc_trimming/trimmed_reads/ERR2198703_2.fastq.gz
-rw-r--r-- 1 kolodisner bhurwitz 1924374219 Mar 29 12:43 /xdisk/bhurwitz/virus_hunting/kolodisner/02_qc_trimming/trimmed_reads/ERR2198704_1.fastq.gz
-rw-r--r-- 1 kolodisner bhurwitz 1948849294 Mar 29 12:43 /xdisk/bhurwitz/virus_hunting/kolodisner/02_qc_trimming/trimmed_reads/ERR2198704_2.fastq.gz
-rw-r--r-- 1 kolodisner bhurwitz 1896715780 Mar 29 12:43 /xdisk/bhurwitz/virus_hunting/kolodisner/02_qc_trimming/trimmed_reads/ERR2198705_1.fastq.gz
-rw-r--r-- 1 kolodisner bhurwitz 1914595566 Mar 29 12:43 /xdisk/bhurwitz/virus_hunting/kolodisner/02_qc_trimming/trimmed_reads/ERR2198705_2.fastq.gz
untrimmed:
-rw-r--r-- 1 kolodisner bhurwitz 2394964413 Mar 25 13:40 /xdisk/bhurwitz/virus_hunting

## Step 4 (optional) QC Final Check

If you have any doubts about the trimming process, you can always run fastqc on the trimmed data and double check that you see all "green". You can check the fastqc files using Jupyter to check for any failures or other warnings.

In [28]:
# Here we are re-running fastqc on the trimmed data
my_code = '''#!/bin/bash
#SBATCH --ntasks=1
#SBATCH --nodes=1             
#SBATCH --time=10:00:00   
#SBATCH --partition=standard
#SBATCH --account=bhurwitz
#SBATCH --array=0-2                         
#SBATCH --output=Job-fastqc-trim-%a.out
#SBATCH --cpus-per-task=1                  
#SBATCH --mem=4G                           

pwd; hostname; date

source ./config.sh
names=($(cat $XFILE_DIR/$XFILE))
TRIM_DIR="${WORK_DIR}/trimmed_reads"

apptainer run /contrib/singularity/shared/bhurwitz/fastqc-0.11.9.sif fastqc \
    $TRIM_DIR/${names[${SLURM_ARRAY_TASK_ID}]}_*.fastq*

mkdir ~/check_fastqc_trimmed2
cp $TRIM_DIR/${names[${SLURM_ARRAY_TASK_ID}]}_*_fastqc.html ~/check_fastqc_trimmed2 

'''

with open('run_fastqc_trim.sh', mode='w') as file:
    file.write(my_code)

In [41]:
!sbatch ./run_fastqc_trim.sh

Submitted batch job 3005522


In [48]:
# check to see if your job is finished running
!squeue --user=$netid

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           3003641  standard  jupyter kolodisn  R    1:47:53      1 i16n2


Once your job completes, you can look at the *.html files in your home directory in ~/check_fastqc_trimmed. 

## Final Step
Copy your notebook to the current working directory

In [ ]:
cp ~/02_qc_trimming.ipynb $work_dir